# Principal Component Analysis and Constant Shift Embedding

Güney Işık Tombak \\ Bogazici AI



## A) Principal Component Analysis

#### 1) Eigenvalues and Eigenvectors

Describe briefly eigenvalues $\lambda_i$ and eigenvectors $v_i$ of a square matrix $A$. 
Write down also the mathematical definition in terms of the variables given above. 
Realize that you should use $\LaTeX$ notation.

Then use `numpy.linalg.eig` to find the eigenvalues and eigenvectors of matrices `A`, `B`, `C`, `D`, and `E`. and print them. Also print the dot product of eigenvectors. What do you observe and what does it mean mathematically? When do we see an eigenvalue of $0$? What is the affect of scalar multiplication on eigenvalues and eigenvectors? What is the affect of scalar addition to diagonal entries on eigenvalues and eigenvectors? Are the eigenvectors unique (other than their negative) for all matrices? Comment on these. 

Your answer here

In [ ]:
import numpy as np
from numpy.linalg import eig

np.random.seed(42)

A = np.array([[0, 2], 
              [2, 3]])

B = np.array([[1, 2], 
              [2, 4]])

C = np.array([[0, 0], 
              [0, 0]])

D = 5*A

E = A + 5*np.eye(2)

# Your answer here

#### 2) Principal Component Analysis

For the Iris dataset given the cell below, use eigenvalue decomposition to the covariance matrix of the dataset. Then, compare eigenvectors with the SciKit-Learn's PCA results. For the number of components, try all numbers from 1 to 4. By using `%timeit`, compare the duration of PCA runs with different number of components.

After this, visualize the samples in 2D and 3D where the targets are depicted as colors. To achieve this, use PCA with 2 and 3 components.

Comment on your results. 


In [2]:
import pandas as pd
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

# load dataset into Pandas DataFrame
df = pd.read_csv(url, names=['sepal length','sepal width',
                             'petal length','petal width','target'])

from sklearn.preprocessing import StandardScaler
features = ['sepal length', 'sepal width', 'petal length', 'petal width']
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
y = df.loc[:,['target']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [ ]:
# Your answer here

Your answer here

## B) Constant Shift Embedding

#### Prologue 

In our standard understanding of samples, we try to fit them into an Euclidean (in general, Hilbert) space. However, the data is not given fitting to our everyday understanding of distance, which is 

$$d(x, y) = \|x - y\| = \sqrt{\sum_{k=1}^d |x_k - y_k|}.$$

More generally, the spaces that we encounter frequently are *metric*, for which the following conditions are satisfied:

1.   Identity: $d(x,x) = 0$
2.   Non-negativity: $\forall (x,y): d(x,y) \geq 0$
3.   Symmetry: $d(x,y) = d(y,x)$
4.   Triangle Inequality: $d(x,y) \leq d(x,z) + d(y,z)$

In some cases, we might have a matrix of size $n \times n$ containing the distances between pairwise objects. More interestingly, these distances might not satisfy the metric space conditions. In this case, we can express distances in a so-called dissimilarity matrix $D$ and similarities in an, again so-called similarity matrix $S$. Without a topological embedding, all these relations are can be conceptualized using **Graph Theory**.

**All the  parts should be implemented on the similarity matrix computed in the cell below.**

If you are not happy with the results, you can try different transforms that you can find in this [link](https://cran.r-project.org/web/packages/linkprediction/vignettes/proxfun.html)

References: 

1.   Main article: [Optimal cluster preserving embedding of nonmetric proximity data](https://ieeexplore.ieee.org/document/1251147)
2.   Dataset: [email-Eu-core](https://snap.stanford.edu/data/email-Eu-core.html)


In [9]:
# Data Read

import scipy.linalg as la
import numpy as np

NUM_NODES = 1005

def adj2sim(A, betaMult=0.7):
    
    # Method: Katz Centrality
    # Katz, L. A new status index derived from sociometric analysis. 
    # Psychometrika 18, 39–43 (1953).
    
    _, l, _ = la.svd(A)
    beta = betaMult/l.max()
    I = np.eye(A.shape[0])
    S = np.linalg.inv(I - beta*A) - I

    return S

# initialize data matrix which will be an adjacency matrix
DATA = np.zeros((NUM_NODES, NUM_NODES))

# fill out the symmetric adjacency matrix
with open("email-Eu-core.txt") as file:
    for line in file:
        pair = [int(x) for x in line.split()]
        DATA[pair[0], pair[1]] = 1
        DATA[pair[1], pair[0]] = 1

S_DATA = adj2sim(DATA)

[[1.34328836e-02 1.13448802e-02 1.55000413e-03 ... 9.93125205e-05
  7.12774878e-06 7.04769355e-06]
 [1.13448802e-02 1.46394427e-02 1.45482701e-03 ... 1.02830753e-04
  1.21160727e-05 6.33376050e-06]
 [1.55000413e-03 1.45482701e-03 2.17420839e-02 ... 1.42356611e-05
  2.45589376e-05 1.32850899e-04]
 ...
 [9.93125205e-05 1.02830753e-04 1.42356611e-05 ... 8.27330233e-05
  1.47160780e-07 6.18092008e-08]
 [7.12774878e-06 1.21160727e-05 2.45589376e-05 ... 1.47160780e-07
  8.35639124e-05 9.95162704e-08]
 [7.04769355e-06 6.33376050e-06 1.32850899e-04 ... 6.18092008e-08
  9.95162704e-08 8.34351012e-05]]


#### 1) Similarity and Dissimilarity

Implement the definitions below as functions for similarity $S$ and dissimilarity $D$ matrices. Also answer the questions regarding them.

a) Psychological: $S_{ij} = e^{-D_{ij}/\Delta}$

Derive the inverse formula. Using `numpy.allclose` show that the inverse operation is true. What is the effect of $\Delta$? Comment on the extreme cases $\Delta = 0$ and  $\Delta \rightarrow \infty$.

b) Linear: $S_{ij} = max(D) - D_{ij}$

Derive the inverse formula. Using `numpy.allclose` show that the inverse operation is true. What could be done for normalization of this method?

c) Euclidean: $D_{ij} = S_{ii} + S_{jj} - 2S_{ij}$

Why this one called as Euclidean distance? Please mathematically explain. 
*Hint:* $S_{ij} \iff x_i^\top x_j$

**Note:** Any similarity or dissimilarity matrix should be symmetric, i.e. $S_{ij} = S_{ji}$ and $D_{ij} = D_{ji}$. Therefore, implement the symmetrization as a seperate function and use it at the very beginning of the code.

$$A^{s}_{ij} = \frac{A_{ij} + A_{ji}}{2}$$

In [ ]:
def symmetrize(A):
    pass
    # Your answer here

def psych_D2S(D, del=1):
    pass
    # Your answer here


def psych_S2D(S, del=1):
    pass
    # Your answer here


def linear_D2S(D):
    pass 
    # Your answer here

    
def linear_S2D(S):
    pass 
    # Your answer here

def euc_S2D(S):
    pass
    # Your answer here


# Your answer here

Your answer here

#### 2) Euclidean Projection by Centralization and Shift of Similarity Matrix

Explain why the inverse of the Euclidean transform from dissimilarity to similarity is not unique under the definition: $D_{ij} = S_{ii} + S_{jj} - 2S_{ij}$.

One way to achieve uniqueness is using centralization, which is defined for any matrix $V$:

$$A^c = QAQ^\top, \ \ \ Q = I_n - \frac{1_n}{n}$$ where $I_n$ is identity matrix whereas $1_n$ is all 1 matrix, both of size $n \times n$. 

With a series of calculations we achieve

$$S^c_{ij} = - \frac{1}{2} D^c_{ij}$$

and with this we have a unique definition for both sides. 

Now, we can consider $S$ as a covariance matrix such that $S = XX^\top$. Hence, we can represent the $n$ objects in these matrices in an Euclidean space. To achieve these we can use Singular Value Decomposition such that 

$$S = V \Lambda V^\top \iff X = V \Lambda^\frac{1}{2}.$$

Realize that to achieve vectors in Euclidean space, the diagonal matrix $\Lambda$ should be all positive. To make all eigenvalues greater or equal than zero ($\lambda_i \geq 0$), we can subtract the lowest eigenvalue of $S$ from $S$ diagonally.

$$\tilde{S} = S - \lambda_{min}(S).$$

In this part, define a class that contains $D^c$, $S^c$, $\tilde{S}$, and $X$. The class should be initializable using a $D$ or $C$. The steps are:

$D \rightarrow D^s \rightarrow D^c \rightarrow S^c \rightarrow \tilde{S} \rightarrow X$ for starting from a $D$. If S is given, use the standard formula first:  $D_{ij} = S_{ii} + S_{jj} - 2S_{ij}$.

Your answer here

In [ ]:
class Euclidean_Projection():
    def __init__(self, matrix, matrix_type='D'):
        if matrix_type.lower() == 'd':
            D = matrix
        else:
            D = self.s2d(matrix)

        D_s = self.symmetrize(D)
        D_c = self.center(D_s)
        S_c = self.d_c2s_c(D_c)
        S_t = self.diag_shift(S_c)
        X = self.s2x(S_t)

        self.D_c = D_c
        self.S_c = S_c
        self.S_t = S_t
        self.X = X

        # Your answer here
    

#### 3) Dimension Trim

Realize that although we can project the objects into an Euclidean space, the space has to have dimension of number of objects, since $X$ is an $n \times n$ matrix. We can change this using the first $p \leq n$ principal components of $\tilde{S}$. Realize that the eigenvalues are given in descending order, i.e. $\forall \lambda, \lambda_{i+1} \leq \lambda_i$.

Therefore, instead of

$$X = V \Lambda^\frac{1}{2} =\begin{bmatrix}
    \vert & \vert & \vert \\
    v_1   & ...  & v_n \\
    \vert & \vert & \vert
\end{bmatrix} \begin{bmatrix}
    \lambda_1^\frac{1}{2} & 0 & 0 \\
    0   & ...  & 0 \\
    0 & 0 & \lambda_n^\frac{1}{2}
\end{bmatrix}$$

we can do 

$$X_p = V_p \Lambda_p^\frac{1}{2} =\begin{bmatrix}
    \vert & \vert & \vert \\
    v_1   & ...  & v_p \\
    \vert & \vert & \vert
\end{bmatrix}_{n \times p} \begin{bmatrix}
    \lambda_1^\frac{1}{2} & 0 & 0 \\
    0   & ...  & 0 \\
    0 & 0 & \lambda_p^\frac{1}{2} \\
\end{bmatrix}_{p \times p}.$$

Please copy your work from the previous step and write another class method namely `s2x_p(S_t, p=None)`. If $p$ is not given, the code should give the standard $X$ (i.e. $p = n$).

In [ ]:
class Constant_Shift_Embedding():
    
    # Your answer here

#### 4) Visualization and Clustering

Determine the variable $p$ as small as possible satisfying that $\sum_{k=1}^p \lambda_p \geq \frac{9}{10}
\sum_{k=1}^n \lambda_k$. Then using CSE class you previously written, determine the $X_p$ and print the corresponding $D_p$ as a heatmap. Do the same for $p = 2$ and $p = 3$ and also show $X_{p=2}$ $X_{p=2}$ on Euclidean spaces (using scatter). You can use Matplotlib, Seaborn, or any other visualization tool.

After these, use K-Means of Sci-Kit Learn to cluster the samples. Use $K = 1:10$ and determine the number of clusters according to the elbow method.

Comment on your results.

In [ ]:
# Your answer here

Your answer here